In [1]:
!cat container/Dockerfile

# Build an image that can do training and inference in SageMaker
# This is a Python 2 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM ubuntu:18.04

MAINTAINER Amazon AI <sage-learner@amazon.com>


RUN apt-get -y update && apt-get install -y --no-install-recommends \
	 python3 \
	 python3-pip \
         wget \
         python \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*


# Here we get all python packages.
# There's substantial overlap between scipy and numpy that we eliminate by
# linking them together. Likewise, pip leaves the install caches populated which uses
# a significant amount of space. These optimizations save a fair amount of space in the
# image, which reduces start up time.

RUN wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py && \
    pip install numpy==1.16.2 scipy==1.2.1 scikit-learn==0.20.2 pandas flask gevent gunicorn && \
        pip install imbalanced-learn==0.4.3 && \

In [2]:
import boto3
import re
import sagemaker
import os
from sagemaker import get_execution_role
import json
role = get_execution_role()
import pandas as pd
import numpy as np

In [3]:
role

'arn:aws:iam::130159455024:role/service-role/AmazonSageMaker-ExecutionRole-20190428T145852'

In [4]:
import sagemaker as sage
sess = sage.Session()

In [5]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
#region = 'us-east-2'

In [6]:
account

'130159455024'

In [7]:
region

'us-east-1'

In [8]:
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-sagechurn:latest'.format(account, region)
print(image)

130159455024.dkr.ecr.us-east-1.amazonaws.com/sagemaker-sagechurn:latest


In [9]:
model_data='s3://sagemaker-us-east-1-130159455024/Sage_churns/sagemaker-sagechurn-2019-09-24-13-31-15-861/output/model.tar.gz'

In [10]:
model_weights_filename='churn_prediction'

env={
     'model_path':"/opt/ml/model/"+model_weights_filename,
     'bucket':'sagemaker-us-east-1-130159455024'
}
#'model_path':"/opt/ml/model/"+model_weights_filename,

In [11]:
model=sagemaker.model.Model(model_data, image, role=role, env=env, name='churn-inference', sagemaker_session=sess)

In [12]:
endpoint_name='churn-inference' 

In [13]:
# model.deploy(1,'ml.m4.xlarge',endpoint_name=endpoint_name)
model.deploy(1,'ml.m4.4xlarge',endpoint_name=endpoint_name)


Using already existing model: churn-inference


--------------------------------------------------------------------------!

In [14]:
model.model_data

's3://sagemaker-us-east-1-130159455024/Sage_churns/sagemaker-sagechurn-2019-09-24-13-31-15-861/output/model.tar.gz'

In [15]:
valid_location = 's3://sagemaker-us-east-1-130159455024/Sage_churns/valid.csv'

In [16]:
shape=pd.read_csv(valid_location, header=None)

In [17]:
file_name = 'train/train.csv'

In [18]:
with open(file_name, 'r') as file:
    f = file.read()
    k = np.array(f)
#     print(type(b))
#     ne = open('n.txt','wb')
#     ne.write(b)
print(type(k))

<class 'numpy.ndarray'>


In [19]:
endpoint_name

'churn-inference'

In [20]:
client = boto3.client('sagemaker-runtime')
content_type = "text/csv"

In [21]:
response = client.invoke_endpoint(
    EndpointName = endpoint_name, 
    ContentType = content_type,
    Body = f)

In [22]:
response

{'ResponseMetadata': {'RequestId': '0dd22d47-cbc8-400d-806b-95b2ce35f302',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0dd22d47-cbc8-400d-806b-95b2ce35f302',
   'x-amzn-invoked-production-variant': 'AllTraffic',
   'date': 'Tue, 1 Oct 2019 06:59:43 GMT',
   'content-type': 'text/csv; charset=utf-8',
   'content-length': '5998'},
  'RetryAttempts': 0},
 'ContentType': 'text/csv; charset=utf-8',
 'InvokedProductionVariant': 'AllTraffic',
 'Body': <botocore.response.StreamingBody at 0x7efe256a5cf8>}

In [23]:
x=response['Body'].read()
print(x.decode("utf-8"))

0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
1
0
0
0
0
0
0
1
0
1
0
0
1
0
1
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
1
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
1
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
1
1
0
0
0
1
1
0
1
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
1
0
